# Feature Selection Method

이전 노트북에서는 Filter 기반의 통계적 특성을 이용한 변수 선택법에 대해 알아보았다. Filter 기반 변수 선택법은 통계적 측정 방법을 이용하여 피쳐들의 상관관계를 알아내고 관계 없는 변수들을 제거하는 방법이다. Wrapper 방법은 예측 모델을 사용하여 피쳐의 하위 집합(subset)을 계속 테스트하는 방법이다. 이 노트북에서는 Wrapper 기반 Recursive Feature Elimination(RFE)에 대해 알아본다.

## Recursive Feature Elimination(RFE)
RFE는 Wrapper 기반 변수 선택 알고리즘 중 하나다. 재귀적으로 변수를 제거하고 남아 있는 변수에 대한 모델을 구축하는 방식으로 작동한다. 훈련 데이터의 모든 변수부터 시작하여 원하는 수가 남을 때까지 변수를 제거함으로써 변수의 하위 집합을 탐색한다. 이 때, 어떤 변수 조합이 목표 변수에 가장 큰 기여를 하즌지를 식별하기 위해 모델 정확도를 이용한다. 

RFE를 사용할 때 두 가지 중요한 요소를 고려해야한다. 첫 번째는 <strong>선택할 변수의 수</strong>와 <strong>변수 선택을 돕기 위해 사용되는 알고리즘 선택</strong>이다. 

아래 예제는 Logistic Regression과 함께 RFE 알고리즘을 사용하여 상위 5개 변수를 선택한다. 

In [59]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [65]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"
col_names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
df = pd.read_csv(url, names = col_names)
df['class'].value_counts()

0    500
1    268
Name: class, dtype: int64

예측해야 하는 클래스가 2:1 비율로 비교적 불균형하다는 것을 알 수 있다. 따라서, Accuracy와 AUC 지표를 모두 비교하여 모델을 평가한다.

In [61]:
df_array = np.array(df)

split_size = int(0.25 * len(df_array))
np.random.seed(0)
np.random.shuffle(df_array)

X = df_array[:, 0:-1]
y = df_array[:, -1]

X_train = X[:-split_size]
y_train = y[:-split_size]

X_test = X[-split_size:]
y_test = y[-split_size:]

## Modeling (non-RFE)

In [62]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
pred_proba = clf.predict_proba(X_test)[:, 1]
print("Accuracy Score: ", accuracy_score(y_test, pred))
print("AUC Score: ", roc_auc_score(y_test, pred_proba))

Accuracy Score:  0.7604166666666666
AUC Score:  0.7975772259236826


## Modeling (Adjust RFE)

In [63]:
clf = LogisticRegression()
rfe = RFE(clf, 5)

fit = rfe.fit(X_train,y_train)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

Num Features: 5
Selected Features: [ True  True False False False  True  True  True]
Feature Ranking: [1 1 2 3 4 1 1 1]


In [64]:
pred = rfe.predict(X_test)
pred_proba = rfe.predict_proba(X_test)[:, 1]
print("Accuracy Score: ", accuracy_score(pred, y_test))
print("AUC Score: ", roc_auc_score(y_test, pred_proba))

Accuracy Score:  0.765625
AUC Score:  0.803755299818292
